In [ ]:
import os
import pandas as pd

# Define the directory path
directory_path = "../csv2/"

# Define the column names
column_names = ["user", "text", "type", "ts"]

# Create an empty dataframe
df = pd.DataFrame(columns=column_names)

# Loop through all subdirectories starting with "D05"
for subdir, dirs, files in os.walk(directory_path):
    if subdir.startswith(directory_path + "D05"):
        for file in files:
            if file.endswith(".csv"):
                # Read the csv file
                file_path = os.path.join(subdir, file)
                temp_df = pd.read_csv(file_path)
                
                # Check if all required columns are present
                if all(col in temp_df.columns for col in column_names):
                    # Append the dataframe to the main dataframe
                    df = df.append(temp_df[column_names], ignore_index=True)

df['ts'] = pd.to_datetime(df['ts'], unit='s')

# Save the dataframe to a csv file
df.to_csv("merged_msg.csv", index=False)

In [ ]:
import os
import pandas as pd

def merge_csv_files():
    # Take user input for usernames
    username1 = input("Enter username 1: ")
    username2 = input("Enter username 2: ")

    # Read the dms_output.csv file
    dms_data = pd.read_csv('../processed_data/dms_output.csv')

    # Filter the data based on the usernames
    filtered_data = dms_data[(dms_data['member_1'] == username1) & (dms_data['member_2'] == username2)]

    if filtered_data.empty:
        print("No matching records found")
        return

    # Get the folder name from the id column of the matched record
    folder_name = filtered_data.iloc[0]['id']

    # Search for the folder with the same name
    folder_path = None
    for root, dirs, files in os.walk('../csv2/'):
        if folder_name in dirs:
            folder_path = os.path.join(root, folder_name)
            break

    if folder_path is None:
        print("Folder not found")
        return
    # Merge all the CSV files in the folder
    merged_data = pd.DataFrame()
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path, usecols=["user", "text", "type", "ts"])
            df['ts'] = pd.to_datetime(df['ts'], unit='s')  # Convert ts column to human-readable timestamp
            merged_data = pd.concat([merged_data, df])

    # Save the merged data to a new CSV file
    merged_data.to_csv('merged_conversations.csv', index=False)
    print("CSV files merged successfully!")

# Usage example
merge_csv_files()


In [ ]:
# Divide each user in the 'members' column into different columns
members_df = channels_df['members'].str.split(',', expand=True)
members_df = members_df.replace({',':'', '\[':'', '\]':'', '\'':''}, regex=True)

# Rename the columns
members_df.columns = [f"member_{i+1}" for i in range(members_df.shape[1])]

# Concatenate the original DataFrame with the new columns
channels_df = pd.concat([channels_df, members_df], axis=1)

# Display the modified DataFrame
channels_df


In [ ]:
dms_df = pd.read_csv('../csv2/dms.csv')

# Divide each user in the 'members' column into different columns
members_df = dms_df['members'].str.split(',', expand=True)
members_df = members_df.replace({',':'', '\[':'', '\]':'', '\'':''}, regex=True)

# Rename the columns
members_df.columns = [f"member_{i+1}" for i in range(members_df.shape[1])]
members_df['member_2'] = members_df['member_2'].str.replace(' ', '')
members_df = members_df.replace(user_id_to_username)

# Concatenate the original DataFrame with the new columns
dms_df = pd.concat([dms_df, members_df], axis=1)

# # rename user id with username
# for column in dms_df.columns:
#     if column.startswith('member_'):
#         dms_df[column] = dms_df[column].replace(user_id_to_username)

# Display the modified DataFrame
dms_df.to_csv('../processed_data/dms_output.csv', index=False)

In [ ]:
import zipfile
import json
import os
import pandas as pd
import psycopg2
from fastapi import FastAPI, UploadFile

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="your_database_name",
    user="your_username",
    password="your_password"
)

app = FastAPI()

@app.post("/process_zip")
async def process_zip(file: UploadFile):
    # Specify the directory to extract the zip file to
    extract_directory = '../data4/raw/'

    # Save the zip file
    zip_file_path = os.path.join(extract_directory, file.filename)
    with open(zip_file_path, 'wb') as f:
        f.write(await file.read())

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_directory)

    # Get a list of JSON files in the directory
    json_files = [os.path.join(root, f) for root, _, files in os.walk(extract_directory) for f in files if f.endswith('.json')]

    # Process each JSON file and convert it to a Pandas DataFrame
    df_list = []
    for json_file in json_files:
        with open(json_file, encoding='utf-8') as f:
            data = json.load(f)
        df = pd.json_normalize(data)
        df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    df = pd.concat(df_list, ignore_index=True)

    # Save the DataFrame to the PostgreSQL database
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS processed_files (id SERIAL PRIMARY KEY, file_name VARCHAR, file_data JSONB)")
    for index, row in df.iterrows():
        file_name = row['file_name']
        file_data = row.to_json()
        cursor.execute("INSERT INTO processed_files (file_name, file_data) VALUES (%s, %s)", (file_name, file_data))
    conn.commit()
    cursor.close()

    return {"message": "Zip file processed successfully"}

@app.get("/create_user_mapping")
async def create_user_mapping():
    # Read the users.csv file
    csv_file_path = '../csv/users.csv'
    df = pd.read_csv(csv_file_path)

    # Create a dictionary for mapping user_id with username
    user_mapping = df.set_index('user_id')['username'].to_dict()

    # Save the user_mapping to the PostgreSQL database
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS user_mapping (user_id VARCHAR PRIMARY KEY, username VARCHAR)")
    for user_id, username in user_mapping.items():
        cursor.execute("INSERT INTO user_mapping (user_id, username) VALUES (%s, %s)", (user_id, username))
    conn.commit()
    cursor.close()

    return {"message": "User mapping created successfully"}

@app.get("/create_channel_info")
async def create_channel_info():
    # Read the channels.csv file
    csv_file_path = '../csv/channels.csv'
    df = pd.read_csv(csv_file_path)

    # Create a new DataFrame with channel information
    channel_info_df = pd.DataFrame(columns=['channel_name', 'members', 'type'])

    # Iterate through each channel
    for index, row in df.iterrows():
        channel_name = row['channel_name']
        members = row['members']
        channel_type = row['type']

        # Split the members string into a list
        members_list = members.split(',')

        # Create a new row for each member in the channel
        for member in members_list:
            channel_info_df = channel_info_df.append({'channel_name': channel_name, 'members': member, 'type': channel_type}, ignore_index=True)

    # Save the channel information to the PostgreSQL database
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS channel_info (channel_name VARCHAR, members VARCHAR, type VARCHAR)")
    for index, row in channel_info_df.iterrows():
        channel_name = row['channel_name']
        members = row['members']
        channel_type = row['type']
        cursor.execute("INSERT INTO channel_info (channel_name, members, type) VALUES (%s, %s, %s)", (channel_name, members, channel_type))
    conn.commit()
    cursor.close()

    return {"message": "Channel information created successfully"}

@app.get("/get_single_user_messages")
async def get_single_user_messages(user_id: str):
    # Read the messages.csv file
    csv_file_path = '../csv/messages.csv'
    df = pd.read_csv(csv_file_path)

    # Filter the messages for the specified user_id
    filtered_df = df[df['user_id'] == user_id]

    # Save the filtered messages to the PostgreSQL database
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS single_user_messages (user_id VARCHAR, message VARCHAR)")
    for index, row in filtered_df.iterrows():
        user_id = row['user_id']
        message = row['message']
        cursor.execute("INSERT INTO single_user_messages (user_id, message) VALUES (%s, %s)", (user_id, message))
    conn.commit()
    cursor.close()

    return {"message": "Single user messages created successfully"}

@app.get("/get_two_user_messages")
async def get_two_user_messages(user_id1: str, user_id2: str):
    # Read the messages.csv file
    csv_file_path = '../csv/messages.csv'
    df = pd.read_csv(csv_file_path)

    # Filter the messages for the specified user_id1 and user_id2
    filtered_df = df[(df['user_id'] == user_id1) | (df['user_id'] == user_id2)]

    # Save the filtered messages to the PostgreSQL database
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS two_user_messages (user_id VARCHAR, message VARCHAR)")
    for index, row in filtered_df.iterrows():
        user_id = row['user_id']
        message = row['message']
        cursor.execute("INSERT INTO two_user_messages (user_id, message) VALUES (%s, %s)", (user_id, message))
    conn.commit()
    cursor.close()

    return {"message": "Two user messages created successfully"}
